In [1]:
import pandas as pd
from stats import get_all_in_dir

path = 'results'

ModuleNotFoundError: No module named 'stats'

In [ ]:
li = []

for file in get_all_in_dir(path, 'csv'):
    df = pd.read_csv(file, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)